## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped = application_df.drop(['EIN', 'NAME'], axis=1)
dropped.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
dropped.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
dropped['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
#create a copy dataframe to work with
dropped_copy = dropped.copy()

In [8]:
# Application types with under 100 instances will be replaced with 'Other'
# list of types using the variable name `application_types_to_replace`

application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']


# Replace in dataframe
for app in application_types_to_replace:
    dropped_copy['APPLICATION_TYPE'] = dropped_copy['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
dropped_copy['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
v_counts = pd.DataFrame(dropped_copy['CLASSIFICATION'].value_counts())
v_counts

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C2500,1
C2600,1
C2150,1
C4120,1


In [26]:
v_counts.reset_index(inplace=True)
v_counts

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C2500,1
67,C2600,1
68,C2150,1
69,C4120,1


In [33]:
v_counts.dtypes

index             object
CLASSIFICATION     int64
dtype: object

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
dropped_copy['CLASSIFICATION'].value_counts().loc[lambda x: x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1246        2
C1234        2
C1256        2
C1267        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [49]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = []

for index, row in v_counts.iterrows():
    if row["CLASSIFICATION"] < 100:
        classifications_to_replace.append(row["index"])
        
classifications_to_replace

['C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C2300',
 'C7200',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C6000',
 'C1800',
 'C1250',
 'C8200',
 'C1278',
 'C1238',
 'C1237',
 'C1235',
 'C7210',
 'C1720',
 'C4100',
 'C2400',
 'C1600',
 'C1257',
 'C2710',
 'C1260',
 'C0',
 'C1246',
 'C1234',
 'C1256',
 'C1267',
 'C3200',
 'C1283',
 'C2570',
 'C1732',
 'C1248',
 'C4200',
 'C8210',
 'C1580',
 'C1900',
 'C1728',
 'C1370',
 'C6100',
 'C1570',
 'C3700',
 'C2380',
 'C1236',
 'C1245',
 'C4500',
 'C2170',
 'C5200',
 'C2190',
 'C2561',
 'C2500',
 'C2600',
 'C2150',
 'C4120',
 'C1820']

In [50]:
# Replace in dataframe
for cls in classifications_to_replace:
    dropped_copy['CLASSIFICATION'] = dropped_copy['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
dropped_copy['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [51]:
# Convert categorical data to numeric with `pd.get_dummies`
dum = pd.get_dummies(dropped_copy)
dum

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [53]:
# Split our preprocessed data into our features and target arrays
X = dum.drop('IS_SUCCESSFUL', axis=1)

y = dum['IS_SUCCESSFUL']

In [56]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
X_train

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
10491,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9384,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11614,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
17386,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3844,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,1,333518,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
6265,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11284,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
860,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Compile, Train and Evaluate the Model

In [65]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()



# First hidden layer
number_input_features = 50

nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 255       
                                                                 
 dense_1 (Dense)             (None, 8)                 48        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 312
Trainable params: 312
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [67]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 858us/step - loss: 0.6294 - accuracy: 0.6795
Epoch 2/100
804/804 [==============================] - 1s 863us/step - loss: 0.5915 - accuracy: 0.7202
Epoch 3/100
804/804 [==============================] - 1s 850us/step - loss: 0.5779 - accuracy: 0.7248
Epoch 4/100
804/804 [==============================] - 1s 864us/step - loss: 0.5717 - accuracy: 0.7270
Epoch 5/100
804/804 [==============================] - 1s 848us/step - loss: 0.5676 - accuracy: 0.7295
Epoch 6/100
804/804 [==============================] - 1s 871us/step - loss: 0.5643 - accuracy: 0.7288
Epoch 7/100
804/804 [==============================] - 1s 877us/step - loss: 0.5623 - accuracy: 0.7278
Epoch 8/100
804/804 [==============================] - 1s 879us/step - loss: 0.5609 - accuracy: 0.7281
Epoch 9/100
804/804 [==============================] - 1s 845us/step - loss: 0.5598 - accuracy: 0.7285
Epoch 10/100
804/804 [==============================] - 1s 885us/step - l

In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5544 - accuracy: 0.7273 - 244ms/epoch - 912us/step
Loss: 0.5543683767318726, Accuracy: 0.7273469567298889


In [69]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")